# Profitable App Profiles for the App Store and Google Play Markets
Our aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets. We're working as data analysts for a company that builds Android and iOS mobile apps, and our job is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

# Opening and Exploring the Data
As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our purpose:

* [A data set](https://www.kaggle.com/lava18/google-play-store-apps/home) containing data about approximately ten thousand Android apps from Google Play
* [A data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) containing data about approximately seven thousand iOS apps from the App Store
Let's start by opening the two data sets and then continue with exploring the data.

In [1]:
from csv import reader

with open("data_sets/AppleStore.csv", encoding='utf8') as file_opened:
    file_readed = list(reader(file_opened))
    apple_data = file_readed[1:]
    apple_headers = file_readed[0]

In [2]:
from csv import reader

with open("data_sets/googleplaystore.csv", encoding='utf8')as file_opened:
    file_readed = list(reader(file_opened))
    google_data = file_readed[1:]
    google_headers = file_readed[0]

To make it easier to explore the two data sets, we'll first write a function named explore_data() that we can use repeatedly to explore rows in a more readable way. We'll also add an option for our function to show the number of rows and columns for any data set.

In [3]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print("%s\n" % row)

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

To make it easier to explore the two data sets, we'll first write a function named explore_data() that we can use repeatedly to explore rows in a more readable way. We'll also add an option for our function to show the number of rows and columns for any data set.

In [4]:
explore_data(google_headers, 0, len(google_headers), False)
explore_data(google_data, 0, 1, True)

App

Category

Rating

Reviews

Size

Installs

Type

Price

Content Rating

Genres

Last Updated

Current Ver

Android Ver

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']

Number of rows: 10841
Number of columns: 13


We see that the Google Play data set has 10841 apps and 13 columns. At a quick glance, the columns that might be useful for the purpose of our analysis are 'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price', and 'Genres'.

Now let's take a look at the App Store data set.

In [5]:
explore_data(apple_headers, 0, len(apple_headers), False)
explore_data(apple_data, 0, 1, True)



id

track_name

size_bytes

currency

price

rating_count_tot

rating_count_ver

user_rating

user_rating_ver

ver

cont_rating

prime_genre

sup_devices.num

ipadSc_urls.num

lang.num

vpp_lic

['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']

Number of rows: 7197
Number of columns: 17


We have 7197 iOS apps in this data set, and the columns that seem interesting are: 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', and 'prime_genre'. Not all column names are self-explanatory in this case, but details about each column can be found in the data set [documentation](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home).

# Deleting Wrong Data
The Google Play data set has a dedicated [discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion) , and we can see that [one of the discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) outlines an error for row 10472. Let's print this row and compare it against the header and another row that is correct.


In [6]:
print(google_data[10472])  # incorrect row
print('\n')
print(google_headers)  # header
print('\n')
print(google_data[0])      # correct row

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


The row 10472 corresponds to the app Life Made WI-Fi Touchscreen Photo Frame, and we can see that the rating is 19. This is clearly off because the maximum rating for a Google Play app is 5. As a consequence, we'll delete this row

In [7]:
print(len(google_data))
del google_data[10472]  # don't run this more than once
print(len(google_data))

10841
10840



# Removing Duplicate Entries¶
## Part One
Let's analyze datasets to detect duplicates

In [8]:
def analyze_duplicates(data_set, data_set_name):
    unique_apps = []
    duplicate_apps = []
    for row in data_set:
        app = row[0]
        if app in unique_apps:
            duplicate_apps.append(app)
        else:
            unique_apps.append(app)

    print("%s unique_apps: %s" % (data_set_name, len(unique_apps)))
    print("%s duplicate_apps: %s" % (data_set_name, len(duplicate_apps)))

First let's check the OS data set

In [9]:
analyze_duplicates(apple_data, "OS")

OS unique_apps: 7197
OS duplicate_apps: 0


Luckily the OS data set is unique. Let's check Android data as well

In [10]:
analyze_duplicates(google_data, "Android")

Android unique_apps: 9659
Android duplicate_apps: 1181


As we can see, the Android data set contains 1181 duplicated entries.

Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']

We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. One thing we could do is remove the duplicate rows randomly, but we could probably find a better way.

If you examine the rows we printed two cells above for the Instagram app, the main difference happens on the fourth position of each row, which corresponds to the number of reviews. The different numbers show that the data was collected at different times. We can use this to build a criterion for keeping rows. We won't remove rows randomly, but rather we'll keep the rows that have the highest number of reviews because the higher the number of reviews, the more reliable the ratings.

To do that, we will:

* Create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app
* Use the dictionary to create a new data set, which will have only one entry per app (and we only select the apps with the highest number of reviews)

## Part Two
Let's start by building the dictionary.

In [11]:
reviews_max = {}

for row in google_data:
    name = row[0]
    n_reviews = float(row[3])
    if name in reviews_max.keys():
        if reviews_max[name] < n_reviews:
            reviews_max[name] = n_reviews
    else:
        reviews_max[name] = n_reviews

In a previous code cell, we found that there are 1,181 cases where an app occurs more than once, so the length of our dictionary (of unique apps) should be equal to the difference between the length of our data set and 1,181.

In [12]:
print('Expected length:', len(google_data) - 1181)
print('Actual length:', len(reviews_max))

Expected length: 9659
Actual length: 9659


Now, let's use the reviews_max dictionary to remove the duplicates. For the duplicate cases, we'll only keep the entries with the highest number of reviews. In the code cell below:

* We start by initializing two empty lists, android_clean and already_added.
* We loop through the android data set, and for every iteration:
    * We isolate the name of the app and the number of reviews.
    * We add the current row (app) to the android_clean list, and the app name (name) to the already_cleaned list if:
The number of reviews of the current app matches the number of reviews of that app as described in the reviews_max dictionary; and
The name of the app is not already in the already_added list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for reviews_max[name] == n_reviews, we'll still end up with duplicate entries for some apps.

In [13]:
clean_google_data = []
already_added = []

for row in google_data:
    name = row[0]
    n_reviews = float(row[3])
    if n_reviews == reviews_max[name] and name not in already_added:
        clean_google_data.append(row)
        already_added.append(name)

Now let's quickly explore the new data set, and confirm that the number of rows is 9,659.

In [14]:
explore_data(clean_google_data, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']

Number of rows: 9659
Number of columns: 13


We have 9659 rows, just as expected.

# Removing Non-English Apps
## Part One
If you explore the data sets enough, you'll notice the names of some of the apps suggest they are not directed toward an English-speaking audience. Below, we see a couple of examples from both data sets:

In [15]:
print(apple_data[813][1])
print(apple_data[6731][1])

print(clean_google_data[4412][0])
print(clean_google_data[7940][0])

436672029
1144164707
中国語 AQリスニング
لعبة تقدر تربح DZ


We're not interested in keeping these kind of apps, so we'll remove them. One way to go about this is to remove each app whose name contains a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;, etc.), and other symbols (+, *, /, etc.).

All these characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, and we can take advantage of that to build a function that checks an app name and tells us whether it contains non-ASCII characters.

We built this function below, and we use the built-in ord() function to find out the corresponding encoding number of each character.

In [16]:
def is_english(string):
    
    strint = string.encode('ascii', 'ignore').decode('ascii')
    
    for character in string:
        if ord(character) > 127:
            return False
    
    return True

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


The function seems to work fine, but some English app names use emojis or other symbols (™, — (em dash), – (en dash), etc.) that fall outside of the ASCII range. Because of this, we'll remove useful apps if we use the function in its current form.

In [17]:

print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

print(ord('™'))
print(ord('😜'))

False
False
8482
128540


## Part Two
To minimize the impact of data loss, we'll only remove an app if its name contains non-ASCII characters:

In [18]:
from langdetect import detect
import emoji

def give_emoji_free_text(text):
    all_chars = [str for str in text]
    emoji_list = [c for c in all_chars if c in emoji.UNICODE_EMOJI]
    for emoji_value in emoji_list:
        all_chars.remove(emoji_value)
    
    clean_text = ''.join(all_chars)
    return clean_text.strip()

def is_english(string):
    clean_strint = give_emoji_free_text(string)
    try:
        clean_strint.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('ナビタイム ドライブサポーター - NAVITIMEのカーナビアプリ : 0'))
print(is_english('Instachat 😜'))
print(is_english('自転車ナビ by NAVITIME(ナビタイム) - 自転車のナビができるアプリ : 0'))

True
False
True
False


The function is still not perfect, and very few non-English apps might get past our filter, but this seems good enough at this point in our analysis — we shouldn't spend too much time on optimization at this point.

Below, we use the is_english() function to filter out the non-English apps for both data sets:

In [19]:
google_data_english = []
apple_data_english = []

for app in clean_google_data:
    name = app[0]
    if is_english(name):
        google_data_english.append(app)
        
for app in apple_data:
    name = app[2]
    if is_english(name):
        apple_data_english.append(app)
        
explore_data(google_data_english, 0, 3, True)
print('\n')
explore_data(apple_data_english, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']

['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']

Number of rows: 9282
Number of columns: 13


['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']

['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1']

['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188583', '2822', '3

# Isolating the Free Apps
As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. Below, we isolate the free apps for both our data sets.

In [20]:
google_final = []
apple_final = []

for app in google_data_english:
    price = app[7]
    if price == '0':
        google_final.append(app)
        
for app in apple_data_english:
    price = app[5]
    if price == '0':
        apple_final.append(app)
        
print(len(google_final))
print(len(apple_final))

8554
3020


We're left with 8864 Android apps and 4056 iOS apps, which should be enough for our analysis.

# Most Common Apps by Genre
## Part One
As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

Build a minimal Android version of the app, and add it to Google Play.
If the app has a good response from users, we then develop it further.
If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.
Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.

## Part Two
We'll build two functions we can use to analyze the frequency tables:

* One function to generate frequency tables that show percentages
* Another function that we can use to display the percentages in a descending order

In [21]:
def freq_table(dataset, index):
    tmp_table = {}
    for line in dataset:
        value = line[index]
        if value in tmp_table.keys():
            tmp_table[value] += 1
        else:
            tmp_table[value] = 1
            
    result = {}
    dataset_len = len(dataset)
    for key, tmp_value in tmp_table.items():
        key_frequency_value = tmp_value/dataset_len * 100
        result[key] = key_frequency_value
        
    return result

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

## Part Three
We start by examining the frequency table for the prime_genre column of the App Store data set.

In [22]:
display_table(apple_final, -5)

Games : 59.60264900662252
Entertainment : 7.6158940397351
Photo & Video : 4.966887417218543
Education : 3.80794701986755
Social Networking : 3.1788079470198674
Shopping : 2.4503311258278146
Utilities : 2.218543046357616
Music : 2.0860927152317883
Sports : 2.019867549668874
Health & Fitness : 1.95364238410596
Productivity : 1.6556291390728477
Lifestyle : 1.490066225165563
News : 1.2913907284768211
Travel : 1.0927152317880795
Finance : 1.0927152317880795
Weather : 0.8609271523178808
Food & Drink : 0.8609271523178808
Reference : 0.5298013245033113
Business : 0.49668874172185434
Book : 0.26490066225165565
Medical : 0.1986754966887417
Navigation : 0.16556291390728478
Catalogs : 0.09933774834437085


We can see that among the free English apps, more than a half (55.64%) are games. Entertainment apps are about 8-ish %, followed by photo & video apps, which are close to 5%, followed by social networking apps which amount for 3.52% of the apps in our data set.. Only 3.25% of the apps are designed for education

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Let's continue by examining the _Genres_ and _Category_ columns of the Google Play data set (two columns which seem to be related).

In [23]:
display_table(google_final, 1) # Category

FAMILY : 18.926817862988077
GAME : 9.72644376899696
TOOLS : 8.557400046761748
BUSINESS : 4.687865326163198
PRODUCTIVITY : 3.9396773439326633
LIFESTYLE : 3.904606032265607
FINANCE : 3.7175590367079727
MEDICAL : 3.624035538929156
PERSONALIZATION : 3.3785363572597613
SPORTS : 3.2382511105915364
COMMUNICATION : 3.20317979892448
HEALTH_AND_FITNESS : 3.1213467383680147
PHOTOGRAPHY : 2.9693710544774374
NEWS_AND_MAGAZINES : 2.7706336216974514
SOCIAL : 2.6654196866962825
TRAVEL_AND_LOCAL : 2.2913256955810146
SHOPPING : 2.221183072246902
BOOKS_AND_REFERENCE : 2.1627308861351415
DATING : 1.82370820668693
VIDEO_PLAYERS : 1.7535655833528174
MAPS_AND_NAVIGATION : 1.3327098433481412
FOOD_AND_DRINK : 1.227495908346972
EDUCATION : 1.1573532850128596
ENTERTAINMENT : 0.9469254150105214
LIBRARIES_AND_DEMO : 0.9352349777881692
AUTO_AND_VEHICLES : 0.9235445405658173
HOUSE_AND_HOME : 0.8183306055646482
WEATHER : 0.7832592938975917
EVENTS : 0.701426233341127
PARENTING : 0.6546644844517185
ART_AND_DESIGN : 0.6

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the _Genres_ column:

In [24]:
display_table(google_final, -4) # Genres

Tools : 8.545709609539397
Entertainment : 6.055646481178396
Education : 5.342529810614917
Business : 4.687865326163198
Productivity : 3.9396773439326633
Lifestyle : 3.8929155950432546
Finance : 3.7175590367079727
Medical : 3.624035538929156
Personalization : 3.3785363572597613
Sports : 3.3083937339256484
Communication : 3.20317979892448
Action : 3.1564180500350716
Health & Fitness : 3.1213467383680147
Photography : 2.9693710544774374
News & Magazines : 2.7706336216974514
Social : 2.6654196866962825
Travel & Local : 2.2913256955810146
Shopping : 2.221183072246902
Books & Reference : 2.1627308861351415
Simulation : 2.104278700023381
Arcade : 1.8587795183539864
Dating : 1.82370820668693
Casual : 1.800327332242226
Video Players & Editors : 1.730184708908113
Maps & Navigation : 1.3327098433481412
Food & Drink : 1.227495908346972
Puzzle : 1.1456628477905073
Racing : 1.017068038344634
Strategy : 0.9469254150105214
Role Playing : 0.9469254150105214
Libraries & Demo : 0.9352349777881692
Auto & 

The difference between the _Genres_ and the _Category_ columns is not crystal clear, but one thing we can notice is that the _Genres_ column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the _Category_ column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

# Most Popular Apps by Genre on the App Store
One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the _Installs_ column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the _rating_count_tot_ app.

Below, we calculate the average number of user ratings per app genre on the App Store:

In [25]:
def get_avg_n_ratings_by_genre(dataset, genre_index, rating_index):

    result = []
    genres = freq_table(dataset, genre_index)

    for genre in genres:
        total = 0
        len_genre = 0
        for app in dataset:
            genre_app = app[genre_index]
            if genre_app == genre:
                n_ratings = float(app[rating_index])
                total += n_ratings
                len_genre += 1
        avg_n_ratings = total / len_genre
        result.append((avg_n_ratings, genre))
    
    result.sort()
    for value in result:
        print(value[1], ':', value[0])
        pass
        
get_avg_n_ratings_by_genre(apple_final, -5, 6)



Medical : 612.0
Catalogs : 5195.0
Education : 6099.417391304348
Business : 6839.6
Utilities : 11413.179104477613
Entertainment : 14481.995652173913
Book : 16671.0
Lifestyle : 17848.51111111111
Health & Fitness : 19230.86440677966
Games : 22820.230555555554
Productivity : 22842.22
News : 23382.17948717949
Sports : 25382.114754098362
Finance : 26729.090909090908
Shopping : 28517.72972972973
Photo & Video : 29249.766666666666
Food & Drink : 33333.92307692308
Travel : 34115.57575757576
Weather : 48275.57692307692
Music : 55396.01587301587
Social Networking : 75253.84375
Reference : 84258.25
Navigation : 102592.0


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [27]:
for app in apple_final:
    if app[-5] == 'Navigation':
        print(app[2], ':', app[6]) # print name and number of ratings

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Geocaching® : 12811
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5
Google Maps - Navigation & Transit : 154911


The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.

Reference apps have 84,258 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [32]:
for app in apple_final:
    if app[-5] == 'Reference':
        print(app[2], ':', app[6])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
Merriam-Webster Dictionary : 16849
Google Translate : 26786
Night Sky : 12122
WWDC : 762
Jishokun-Japanese English Dictionary & Translator : 0
VPN Express : 14
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Real Bike Traffic Rider Virtual Reality Glasses : 8


However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.